In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import plotly.express as px
from src.download import clustering_data, bids_data

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project='analytics-dev-333113')

query = f'''
SELECT DISTINCT tc.id AS city_id, 
                -- CONCAT(tc.name, ' (', tc.id, ')') AS city, 
                  tc.name AS city, 
                  CONCAT(tcc.name) AS country
             FROM `indriver-e6e40.ods_geo_config.tbl_city` tc
             JOIN `indriver-e6e40.ods_geo_config.tbl_country` tcc 
             ON tc.country_id = tcc.id'''

names_df = client.query(query).result().to_dataframe()

In [16]:
def euclidean_distance(row, x0, y0):
    return np.sqrt((row['order2bid'] - x0)**2 + (row['bid2done'] - y0)**2)

In [44]:
def get_color_for_alpha(alpha_str, cmap):
    """
    Находит цвет в словаре color_map по первой части ключа (до запятой).
    Предполагается, что для одного и того же alpha_str все соответствующие
    значения цвета будут одинаковыми (как в вашем примере).
    """
    # Формируем префикс, который должен быть в начале ключа
    # Добавляем запятую, чтобы '0.1' не совпало с '0.15'
    prefix_to_match = alpha_str + ","
    for key, value in cmap.items():
        if key.startswith(prefix_to_match):
            return value # Возвращаем значение первого найденного совпадения
    return None # Если совпадений не найдено

# Получаем цвет
# found_color = get_color_for_alpha(search_alpha_value, color_map)

In [69]:
ok_types_list = ['auto_econom', 'auto_econom_without_option_4_pass', 'auto_econom_karaganda', 'Ride A/C', 
                 'auto_econom_dublicate', 'auto_econom_nocomment', 'auto_econom_wheelchair', 'auto_econom_Ukraine']

### Veracruz Proxy

In [373]:
start_date='2025-02-01'
stop_date='2025-02-28'
min_trips_threshold = 300000

df = pd.read_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv')

start_date = '2025-01-29'
stop_date = '2025-02-26'
MIN_COMPLETED_ORDERS = 300000

df_bids = pd.read_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv')

In [374]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        # color='order_type',
        color = 'highlight_group',
        # symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city': True,
            'country': True,
            'city_id': True,
            'distance_to_point': ':.4f',
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'city': 'City',
            'country': 'Country',
            'distance_to_point': 'Distance Euclidean',
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=800,
        width=900
    )

    fig.show()

data = df.copy()
data = data[data['order_type'].isin(ok_types_list)]
data['order_type'] = 'Econom_My'

data = pd.merge(names_df, data, on='city_id', how='right')

veracruz_id = [4180] 
data['distance_to_point'] = data.apply(lambda row: euclidean_distance(row, 
                                                                      data[data['city_id'] == 4180]['order2bid'].values[0],
                                                                      data[data['city_id'] == 4180]['bid2done'].values[0]), axis=1)
veracruz_candidates = data[data['distance_to_point']<=0.06]
data['highlight_group'] = data.apply(lambda row: 'Veracruz' if row['city_id'] in veracruz_id else 
                                     ('Candidates' if row['city_id'] in veracruz_candidates['city_id'].values else 'Normal'), axis=1)

cluster_plot(data)

In [375]:
veracruz_candidates

,city_id,city,country,order_type,total_orders,total_rides,bid2done,order2bid,distance_to_point
20,5548,Panama,Panama,Econom_My,2023201,1300972,0.740160,0.868767,0.052446
30,4236,Hermosillo,Mexico,Econom_My,1568533,1012111,0.748066,0.862570,0.043321
33,4515,Puerto Vallarta,Mexico,Econom_My,1431276,901712,0.741758,0.849341,0.049142
35,4244,Cartagena,Colombia,Econom_My,1340946,842049,0.732413,0.857373,0.058286
50,5504,Cochabamba,Bolivia,Econom_My,1038337,670274,0.743658,0.868042,0.048883
53,4271,Iquique y Alto Hospicio,Chile,Econom_My,903189,577805,0.762708,0.838772,0.032044
55,4180,Veracruz,Mexico,Econom_My,885734,598394,0.790628,0.854499,0.000000
68,4381,Monclova,Mexico,Econom_My,638564,447277,0.834670,0.839184,0.046628


In [376]:
df_filtered = df_bids.copy()
df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS]
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)
df_filtered = df_filtered[df_filtered['order_type'].isin(ok_types_list)]
# df_filtered['order_type'] = 'Econom_My'

default_category = 'Other' 
category_mappings = {
    'Veracruz': [4180],
    'Candidates': veracruz_candidates['city_id'].values.tolist(),
} 
processed_mappings = {key: set(value) for key, value in category_mappings.items()}

def assign_value_based_on_id(current_id):
    for value_to_assign, id_set in processed_mappings.items():
        if current_id in id_set:
            return value_to_assign
    return default_category 

df_filtered['highlight_status'] = df_filtered['city_id'].apply(assign_value_based_on_id)

In [377]:
condition1 = (df_filtered['highlight_status'] == 'Veracruz')
condition2 = (df_filtered['bad_bids_share'] >= 0.099)
condition3 = (df_filtered['bad_bids_share'] <= 0.210)

df_filtered = df_filtered[condition1 | (condition2 & condition3)]

In [378]:
df_filtered = df_filtered.sort_values(by='alpha_value', ascending=True)

In [379]:
if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Veracruz': '#636efa', '0.1, Other': '#636efa', '0.1, Candidates': '#636efa', '0.125, Veracruz': '#EF553B', '0.125, Other': '#EF553B', '0.125, Candidates': '#EF553B', '0.15, Veracruz': '#00cc96', '0.15, Other': '#00cc96', '0.15, Candidates': '#00cc96', '0.2, Veracruz': '#ab63fa', '0.2, Other': '#ab63fa', '0.2, Candidates': '#ab63fa', '0.25, Veracruz': '#FFA15A', '0.25, Other': '#FFA15A', '0.25, Candidates': '#FFA15A', '0.3, Veracruz': '#19d3f3', '0.3, Other': '#19d3f3', '0.35, Veracruz': '#FF6692', '0.35, Other': '#FF6692', '0.4, Veracruz': '#B6E880', '0.4, Other': '#B6E880'}


In [380]:
final_list = [
    {'city_id': 4381, 'alpha_value_numeric': 0.1},
    {'city_id': 4271, 'alpha_value_numeric': 0.15},
    {'city_id': 4244, 'alpha_value_numeric': 0.2},
    {'city_id': 5504, 'alpha_value_numeric': 0.2},
    {'city_id': 5548, 'alpha_value_numeric': 0.2},
]

final_list_df = pd.DataFrame(final_list)
final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
selected_rows_df_veracrus = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
selected_rows_df_veracrus['highlight_status'] = 'Veracruz_Cluster'
selected_rows_df_veracrus = pd.merge(names_df, selected_rows_df_veracrus, on='city_id', how='right')
selected_rows_df_veracrus

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,4381,Monclova,Mexico,auto_econom,220.618175,0.10,1179737.0,138486.0,0.117387,459629.0,19135.0,0.041631,0.1,Veracruz_Cluster
1,4271,Iquique y Alto Hospicio,Chile,auto_econom,200.416556,0.15,1901412.0,253299.0,0.133216,594691.0,26782.0,0.045035,0.15,Veracruz_Cluster
2,4244,Cartagena,Colombia,auto_econom_nocomment,270.369901,0.20,3142921.0,577197.0,0.183650,869506.0,48611.0,0.055906,0.2,Veracruz_Cluster
3,5548,Panama,Panama,auto_econom,270.392303,0.20,4609340.0,692657.0,0.150272,1348288.0,51635.0,0.038297,0.2,Veracruz_Cluster
4,5504,Cochabamba,Bolivia,auto_econom,208.292639,0.20,2315121.0,362965.0,0.156780,671074.0,34667.0,0.051659,0.2,Veracruz_Cluster


### Puerto Proxy

In [381]:
start_date='2025-02-01'
stop_date='2025-02-28'
min_trips_threshold = 300000

df = pd.read_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv')

start_date = '2025-01-29'
stop_date = '2025-02-26'
MIN_COMPLETED_ORDERS = 300000

df_bids = pd.read_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv')

In [382]:
data = df.copy()
data = data[data['order_type'].isin(ok_types_list)]
# data['order_type'] = 'Econom_My'

data = pd.merge(names_df, data, on='city_id', how='right')

puerto_id = [4515] 
data['distance_to_point'] = data.apply(lambda row: euclidean_distance(row, 
                                                                      data[data['city_id'] == 4515]['order2bid'].values[0],
                                                                      data[data['city_id'] == 4515]['bid2done'].values[0]), axis=1)
puerto_candidates = data[data['distance_to_point']<=0.065]
data['highlight_group'] = data.apply(lambda row: 'Puerto' if row['city_id'] in puerto_id else 
                                     ('Candidates' if row['city_id'] in puerto_candidates['city_id'].values else 'Normal'), axis=1)

cluster_plot(data)

In [383]:
puerto_candidates

,city_id,city,country,order_type,total_orders,total_rides,bid2done,order2bid,distance_to_point
6,4257,Trujillo,Peru,auto_econom,3200333,2159515,0.749690,0.900077,0.051352
17,4258,Arequipa,Peru,auto_econom,2408542,1473627,0.708279,0.863832,0.036480
19,4194,Merida,Mexico,auto_econom,2130646,1197706,0.692955,0.811211,0.061933
20,5548,Panama,Panama,auto_econom,2023201,1300972,0.740160,0.868767,0.019491
23,5512,Santa Cruz de la Sierra,Bolivia,auto_econom,1753116,1044317,0.688113,0.865689,0.056080
29,4227,Ciudad Juarez,Mexico,auto_econom,1593303,949098,0.691606,0.861299,0.051558
30,4236,Hermosillo,Mexico,auto_econom,1568533,1012111,0.748066,0.862570,0.014656
31,4231,San Luis Potosi,Mexico,auto_econom,1532299,879438,0.706988,0.811801,0.051169
33,4515,Puerto Vallarta,Mexico,auto_econom,1431276,901712,0.741758,0.849341,0.000000
35,4244,Cartagena,Colombia,auto_econom_nocomment,1340946,842049,0.732413,0.857373,0.012322


In [384]:
df_filtered = df_bids.copy()
df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS]
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)
df_filtered = df_filtered[df_filtered['order_type'].isin(ok_types_list)]
# df_filtered['order_type'] = 'Econom_My'

default_category = 'Other' 
category_mappings = {
    'Puerto': [4515],
    'Candidates': puerto_candidates['city_id'].values.tolist(),
} 
processed_mappings = {key: set(value) for key, value in category_mappings.items()}

def assign_value_based_on_id(current_id):
    for value_to_assign, id_set in processed_mappings.items():
        if current_id in id_set:
            return value_to_assign
    return default_category 

df_filtered['highlight_status'] = df_filtered['city_id'].apply(assign_value_based_on_id)

In [385]:
condition1 = (df_filtered['highlight_status'] == 'Puerto')
condition2 = (df_filtered['bad_bids_share'] >= 0.200)
condition3 = (df_filtered['bad_bids_share'] <= 0.333)

df_filtered = df_filtered[condition1 | (condition2 & condition3)]

In [386]:
df_filtered = df_filtered.sort_values(by='alpha_value', ascending=True)

In [387]:
if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, Puerto': '#636efa', '0.1, Candidates': '#636efa', '0.125, Other': '#EF553B', '0.125, Puerto': '#EF553B', '0.125, Candidates': '#EF553B', '0.15, Other': '#00cc96', '0.15, Puerto': '#00cc96', '0.15, Candidates': '#00cc96', '0.2, Other': '#ab63fa', '0.2, Puerto': '#ab63fa', '0.2, Candidates': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, Puerto': '#FFA15A', '0.25, Candidates': '#FFA15A', '0.3, Other': '#19d3f3', '0.3, Puerto': '#19d3f3', '0.35, Other': '#FF6692', '0.35, Puerto': '#FF6692', '0.4, Puerto': '#B6E880'}
Мало данных (нужно >= 2, есть 1) для OLS alpha_value_str = 0.4.


In [388]:
final_list = [
    {'city_id': 4227, 'alpha_value_numeric': 0.125},
    {'city_id': 235, 'alpha_value_numeric': 0.15},
    {'city_id': 228, 'alpha_value_numeric': 0.15},
    {'city_id': 4227, 'alpha_value_numeric': 0.15},
    {'city_id': 262, 'alpha_value_numeric': 0.15},
    {'city_id': 4240, 'alpha_value_numeric': 0.15},
    {'city_id': 4194, 'alpha_value_numeric': 0.20},
    {'city_id': 4231, 'alpha_value_numeric': 0.20},
]

final_list_df = pd.DataFrame(final_list)
final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
selected_rows_df_puerto = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
selected_rows_df_puerto['highlight_status'] = 'Puerto_Cluster'
selected_rows_df_puerto = pd.merge(names_df, selected_rows_df_puerto, on='city_id', how='right')
selected_rows_df_puerto

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,4227,Ciudad Juarez,Mexico,auto_econom,311.244127,0.125,3771375.0,1071000.0,0.283981,974039.0,134341.0,0.137922,0.125,Puerto_Cluster
1,262,Павлодар,Казахстан,auto_econom,208.435669,0.150,1216634.0,303646.0,0.249579,314198.0,34024.0,0.108288,0.15,Puerto_Cluster
2,235,Тараз,Казахстан,auto_econom,236.561399,0.150,1381775.0,405628.0,0.293556,338141.0,47718.0,0.141119,0.15,Puerto_Cluster
3,4227,Ciudad Juarez,Mexico,auto_econom,311.244127,0.150,3771375.0,931071.0,0.246878,974039.0,119546.0,0.122732,0.15,Puerto_Cluster
4,4240,Mazatlan,Mexico,auto_econom,251.085345,0.150,1754872.0,440479.0,0.251003,455620.0,46709.0,0.102517,0.15,Puerto_Cluster
5,228,Семей,Казахстан,auto_econom,201.728708,0.150,2106598.0,599530.0,0.284596,511413.0,68088.0,0.133137,0.15,Puerto_Cluster
6,4231,San Luis Potosi,Mexico,auto_econom,275.777859,0.200,3088359.0,651942.0,0.211097,904724.0,89455.0,0.098875,0.2,Puerto_Cluster
7,4194,Merida,Mexico,auto_econom,272.297612,0.200,4746830.0,980241.0,0.206504,1247554.0,127517.0,0.102214,0.2,Puerto_Cluster


### 4Ilia

In [421]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        # color='order_type',
        color = 'highlight_group',
        # symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city': True,
            'country': True,
            'city_id': True,
            # 'distance_to_point': ':.4f',
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'city': 'City',
            'country': 'Country',
            'distance_to_point': 'Distance Euclidean',
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=800,
        width=900
    )

    fig.show()

In [437]:
start_date='2025-02-01'
stop_date='2025-02-28'
min_trips_threshold = 300000

df = pd.read_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv')

start_date = '2025-01-29'
stop_date = '2025-02-26'
MIN_COMPLETED_ORDERS = 300000

df_bids = pd.read_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv')

In [438]:
# new_rows = [{'city_id': 4538,
#            'order_type': 'auto_econom',
#            'total_orders': 530970,
#            'total_rides': 242016,
#            'bid2done': 0.55843574490864867,
#            'order2bid': 0.81620807201913481},
#            {'city_id': 4538,
#            'order_type': 'moto_econom',
#            'total_orders': 220988,
#            'total_rides': 105407,
#            'bid2done': 0.603280621326328,
#            'order2bid': 0.79064474089090808}]
# df_new_rows = pd.DataFrame(new_rows)
# df = pd.concat([df, df_new_rows], ignore_index=True)

# df_bids = pd.concat([df_bids, 
#                      pd.read_csv(f'data/df_bids_4538.csv')], ignore_index=True)


In [440]:
data = df.copy()
data = data[data['order_type'].isin(ok_types_list)]
# data['order_type'] = 'Econom_My'

data = pd.merge(names_df, data, on='city_id', how='right')

ilya_ids = [4231, 4263] 
# data['distance_to_point'] = data.apply(lambda row: euclidean_distance(row, 
#                                                                       data[data['city_id'] == 4515]['order2bid'].values[0],
#                                                                       data[data['city_id'] == 4515]['bid2done'].values[0]), axis=1)
# puerto_candidates = data[data['distance_to_point']<=0.065]
data['highlight_group'] = data.apply(lambda row: 'Ilya_ids' if row['city_id'] in ilya_ids else 'Normal', axis=1)

cluster_plot(data)

In [441]:
data[data['highlight_group'] == 'Ilya_ids']

,city_id,city,country,order_type,total_orders,total_rides,bid2done,order2bid,highlight_group
31,4231,San Luis Potosi,Mexico,auto_econom,1532299,879438,0.706988,0.811801,Ilya_ids
48,4263,Antofagasta,Chile,auto_econom,1058172,685999,0.735934,0.880903,Ilya_ids


In [444]:
df_filtered = df_bids.copy()
df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS]
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)
df_filtered = df_filtered[df_filtered['order_type'].isin(ok_types_list)]
# df_filtered['order_type'] = 'Econom_My'

default_category = 'Other' 
category_mappings = {
    # 'Puerto': [4515],
    'Candidates': ilya_ids,
} 
processed_mappings = {key: set(value) for key, value in category_mappings.items()}

def assign_value_based_on_id(current_id):
    for value_to_assign, id_set in processed_mappings.items():
        if current_id in id_set:
            return value_to_assign
    return default_category 

df_filtered['highlight_status'] = df_filtered['city_id'].apply(assign_value_based_on_id)

In [448]:
df_filtered[df_filtered['city_id'].isin(ilya_ids)]

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
3984,4231,auto_econom,275.777859,0.100,3088359.0,1276880.0,0.413449,904724.0,207527.0,0.229382,0.1,Candidates
3985,4231,auto_econom,275.777859,0.125,3088359.0,1144801.0,0.370683,904724.0,185362.0,0.204882,0.125,Candidates
3986,4231,auto_econom,275.777859,0.150,3088359.0,965854.0,0.312740,904724.0,150723.0,0.166596,0.15,Candidates
3987,4231,auto_econom,275.777859,0.200,3088359.0,651942.0,0.211097,904724.0,89455.0,0.098875,0.2,Candidates
3988,4231,auto_econom,275.777859,0.250,3088359.0,384831.0,0.124607,904724.0,48566.0,0.053680,0.25,Candidates
3989,4231,auto_econom,275.777859,0.300,3088359.0,232149.0,0.075169,904724.0,27725.0,0.030645,0.3,Candidates
3990,4231,auto_econom,275.777859,0.350,3088359.0,117206.0,0.037951,904724.0,16014.0,0.017700,0.35,Candidates
3991,4231,auto_econom,275.777859,0.400,3088359.0,61.0,0.000020,904724.0,0.0,0.000000,0.4,Candidates
4520,4263,auto_econom,195.435199,0.100,2685651.0,1178127.0,0.438675,705076.0,122439.0,0.173654,0.1,Candidates
4521,4263,auto_econom,195.435199,0.125,2685651.0,1070312.0,0.398530,705076.0,103290.0,0.146495,0.125,Candidates


In [ ]:
# condition1 = (df_filtered['highlight_status'] == 'Puerto')
# condition2 = (df_filtered['bad_bids_share'] >= 0.200)
# condition3 = (df_filtered['bad_bids_share'] <= 0.333)

# df_filtered = df_filtered[condition1 | (condition2 & condition3)]

In [449]:
if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, Candidates': '#636efa', '0.125, Other': '#EF553B', '0.125, Candidates': '#EF553B', '0.15, Other': '#00cc96', '0.15, Candidates': '#00cc96', '0.2, Other': '#ab63fa', '0.2, Candidates': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, Candidates': '#FFA15A', '0.3, Other': '#19d3f3', '0.3, Candidates': '#19d3f3', '0.35, Other': '#FF6692', '0.35, Candidates': '#FF6692', '0.4, Other': '#B6E880', '0.4, Candidates': '#B6E880'}


In [470]:
final_list = [
    {'city_id': 4231, 'alpha_value_numeric': 0.125},
    {'city_id': 4263, 'alpha_value_numeric': 0.125},
]

final_list_df = pd.DataFrame(final_list)
final_list_df['alpha_value_str'] = final_list_df['alpha_value_numeric'].astype(str)
filter_final_list_df_for_merge = final_list_df[['city_id', 'alpha_value_str']]
selected_rows_df_Ilya = pd.merge(df_filtered, filter_final_list_df_for_merge, on=['city_id', 'alpha_value_str'], how='inner')
selected_rows_df_Ilya['highlight_status'] = 'Ilya_ids_Cluster'
selected_rows_df_Ilya = pd.merge(names_df, selected_rows_df_Ilya, on='city_id', how='right')
selected_rows_df_Ilya

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status,ols_predicted_value,ols_diff,rel_diff
0,4231,San Luis Potosi,Mexico,auto_econom,275.777859,0.125,3088359.0,1144801.0,0.370683,904724.0,185362.0,0.204882,0.125,Ilya_ids_Cluster,0.170505,-0.034377,0.201617
1,4263,Antofagasta,Chile,auto_econom,195.435199,0.125,2685651.0,1070312.0,0.398530,705076.0,103290.0,0.146495,0.125,Ilya_ids_Cluster,0.183315,0.036820,0.200855


### V2

In [453]:
start_date='2025-02-01'
stop_date='2025-02-28'
min_trips_threshold = 300000

df = pd.read_csv(f'data/ClusterCities_{start_date}_{stop_date}.csv')

start_date = '2025-01-29'
stop_date = '2025-02-26'
MIN_COMPLETED_ORDERS = 300000

df_bids = pd.read_csv(f'data/ChooseCities_{start_date}_{stop_date}.csv')

In [454]:
def cluster_plot(df):
    # 2. Используем новую колонку для параметра 'symbol'
    fig = px.scatter(
        df,
        x='order2bid',
        y='bid2done',
        # size='total_orders',
        # color='order_type',
        color = 'highlight_group',
        # symbol='highlight_group',  # Используем новую колонку для символов
        symbol_map={ # Опционально: задаем конкретные символы
            'Highlighted': 'star',
            'Normal': 'circle'
        },
        hover_name='city_id',
        hover_data={
            'city': True,
            'country': True,
            'city_id': True,
            'distance_to_recifeMoto': ':.4f',
            'distance_to_lowright': ':.4f',
            'distance_to_middle': ':.4f',
            'order_type': True,
            'total_orders': ':,',
            'total_rides': ':,',
            'bid2done': ':.4f',
            'order2bid': ':.4f',
            'highlight_group': True # Можно добавить в hover_data для ясности
        },
        title=f'Order2Bid (MR) vs Bid2Done (with Highlighted Cities), {start_date} - {stop_date}',
        labels={
            'city': 'City',
            'country': 'Country',
            'distance_to_point': 'Distance Euclidean',
            'bid2done': 'Bid2Done',
            'order2bid': 'Order2Bid',
            'total_orders': 'Total Orders',
            'total_rides': 'Total Rides',
            'order_type': 'Order Type',
            'city_id': 'City ID',
            'highlight_group': 'Highlight Status' # Лейбл для новой легенды символов
        }
    )

    fig.update_layout(
        legend_title_text='Order Type & Status', # Обновляем заголовок легенды
        height=800,
        width=900
    )

    fig.show()

data = df.copy()
data = data[data['order_type'].isin(ok_types_list)]
data['order_type'] = 'Econom_My'
data = pd.concat([data, df[df['city_id'] == 4373]], ignore_index=True)

data = pd.merge(names_df, data, on='city_id', how='right')

recifeMoto_id = [4373] 
data['distance_to_recifeMoto'] = data.apply(lambda row: euclidean_distance(row, 
                                                                      data[data['city_id'] == 4373]['order2bid'].values[0],
                                                                      data[data['city_id'] == 4373]['bid2done'].values[0]), axis=1)
recifeMoto_candidates = data[data['distance_to_recifeMoto']<=0.06]
recifeMoto_candidates = recifeMoto_candidates[recifeMoto_candidates['city_id'] != 4373]

lowright_id = [5681] 
data['distance_to_lowright'] = data.apply(lambda row: euclidean_distance(row, 
                                                                      data[data['city_id'] == 5681]['order2bid'].values[0],
                                                                      data[data['city_id'] == 5681]['bid2done'].values[0]), axis=1)
lowright_candidates = data[data['distance_to_lowright']<=0.06]
lowright_candidates = pd.concat([lowright_candidates, data[data['city_id'] == 5257]], ignore_index=True)

middle_id = [242] 
data['distance_to_middle'] = data.apply(lambda row: euclidean_distance(row, 
                                                                      data[data['city_id'] == 242]['order2bid'].values[0],
                                                                      data[data['city_id'] == 242]['bid2done'].values[0]), axis=1)
middle_candidates = data[data['distance_to_middle']<=0.05]

data['highlight_group'] = data.apply(lambda row: 'LowRight' if row['city_id'] in lowright_candidates['city_id'].values else 
                                     ('Recife_Moto' if row['city_id'] in recifeMoto_id else
                                     ('Recife_like' if row['city_id'] in recifeMoto_candidates['city_id'].values else 
                                      ('Middle' if row['city_id'] in middle_candidates['city_id'].values else 'Normal'))), axis=1)

cluster_plot(data)

In [455]:
data[data['highlight_group'] != 'Normal']

,city_id,city,country,order_type,total_orders,total_rides,bid2done,order2bid,distance_to_recifeMoto,distance_to_lowright,distance_to_middle,highlight_group
3,4197,Bogota,Colombia,Econom_My,4791749,2139969,0.539464,0.827848,0.034190,0.154692,0.180774,Recife_like
4,4243,Barranquilla,Colombia,Econom_My,4100510,2276723,0.692401,0.801890,0.187298,0.182531,0.032887,Middle
5,4142,São Paulo,Brazil,Econom_My,3307264,1276615,0.498629,0.774130,0.047094,0.221483,0.226064,Recife_like
12,9417,Alger (الجزائر العاصمة),Algeria,Econom_My,2596422,973038,0.476603,0.786317,0.045217,0.223543,0.245728,Recife_like
14,5257,Kolkata,India,Econom_My,2470077,1137113,0.505010,0.911576,0.090945,0.118528,0.233852,LowRight
15,4253,Bucaramanga,Colombia,Econom_My,2450107,1356827,0.707790,0.782411,0.205337,0.206899,0.039245,Middle
16,4349,Lagos,Nigeria,Econom_My,2408761,1022146,0.486340,0.872527,0.055505,0.155092,0.239611,Recife_like
17,4258,Arequipa,Peru,Econom_My,2408542,1473627,0.708279,0.863832,0.206797,0.140626,0.045684,Middle
19,4194,Merida,Mexico,Econom_My,2130646,1197706,0.692955,0.811211,0.187150,0.174506,0.028394,Middle
25,5957,Kathmandu,Nepal,Econom_My,1662371,957207,0.625982,0.919848,0.155654,0.047785,0.137408,LowRight


In [456]:
df_filtered = df_bids.copy()
df_filtered = df_filtered[df_filtered['rides_count'] >= MIN_COMPLETED_ORDERS]
df_filtered['alpha_value_str'] = df_filtered['alpha_value'].astype(str)
df_filtered = df_filtered[df_filtered['order_type'].isin(ok_types_list)]
# df_filtered['order_type'] = 'Econom_My'

default_category = 'Other' 
category_mappings = {
    'LowRight': lowright_candidates['city_id'].values.tolist(),
    'Recife_like': recifeMoto_candidates['city_id'].values.tolist(),
    'Middle': middle_candidates['city_id'].values.tolist(),
} 
processed_mappings = {key: set(value) for key, value in category_mappings.items()}

def assign_value_based_on_id(current_id):
    for value_to_assign, id_set in processed_mappings.items():
        if current_id in id_set:
            return value_to_assign
    return default_category 

df_filtered['highlight_status'] = df_filtered['city_id'].apply(assign_value_based_on_id)

In [457]:
# condition1 = (df_filtered['highlight_status'] == 'Puerto')
# condition2 = (df_filtered['bad_bids_share'] >= 0.200)
# condition3 = (df_filtered['bad_bids_share'] <= 0.333)

# df_filtered = df_filtered[condition1 | (condition2 & condition3)]

In [458]:
df_filtered = df_filtered.sort_values(by='alpha_value', ascending=True)

In [459]:
if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, LowRight': '#636efa', '0.1, Middle': '#636efa', '0.1, Recife_like': '#636efa', '0.125, Other': '#EF553B', '0.125, LowRight': '#EF553B', '0.125, Middle': '#EF553B', '0.125, Recife_like': '#EF553B', '0.15, Other': '#00cc96', '0.15, LowRight': '#00cc96', '0.15, Middle': '#00cc96', '0.15, Recife_like': '#00cc96', '0.2, Other': '#ab63fa', '0.2, LowRight': '#ab63fa', '0.2, Middle': '#ab63fa', '0.2, Recife_like': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, LowRight': '#FFA15A', '0.25, Middle': '#FFA15A', '0.25, Recife_like': '#FFA15A', '0.3, Other': '#19d3f3', '0.3, LowRight': '#19d3f3', '0.3, Middle': '#19d3f3', '0.3, Recife_like': '#19d3f3', '0.35, Other': '#FF6692', '0.35, LowRight': '#FF6692', '0.35, Middle': '#FF6692', '0.35, Recife_like': '#FF6692', '0.4, Other': '#B6E880', '0.4, LowRight': '#B6E880', '0.4, Middle': '#B6E880', '0.4, Recife_like': '#B6E880'}


In [460]:
# This will be the requested table
ols_slopes_for_table = {}

if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График и таблица не будут построены.")
else:
    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        current_slope = np.nan # Default to NaN

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean) # OLS through origin
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                else:
                    slope_val = results.params.iloc[0]
                    if pd.notna(slope_val) and np.isfinite(slope_val):
                        current_slope = slope_val
                        x_range_for_line = np.array([-0.05, 1.0])
                        y_line = current_slope * x_range_for_line
                    else:
                        print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")
        
        ols_slopes_for_table[current_alpha_str] = current_slope

    # --- Start: PandaS TABLE GENERATION ---
    # 2. Calculate OLS predicted value for each point
    def calculate_ols_predicted_value(row):
        alpha_str = row['alpha_value_str']
        bad_bids_val = row['bad_bids_share']
        slope = ols_slopes_for_table.get(alpha_str, np.nan) 
        
        if pd.notna(slope) and pd.notna(bad_bids_val):
            return slope * bad_bids_val
        return np.nan

    df_filtered['ols_predicted_value'] = df_filtered.apply(calculate_ols_predicted_value, axis=1)
    df_filtered['ols_diff'] = df_filtered['ols_predicted_value'] - df_filtered['rides_badbids_share']
    
    # Handle division by zero explicitly
    numerator = abs(df_filtered['ols_predicted_value'] - df_filtered['rides_badbids_share'])
    denominator = df_filtered['ols_predicted_value']

    # Where denominator is 0, result will be NaN. If numerator is also 0, it's NaN.
    # If numerator is non-zero and denominator is 0, it's inf.
    # To avoid inf and set to NaN instead, or a large number, or 0, you can use np.where:
    df_filtered['rel_diff'] = np.where(
        denominator != 0,  # Condition
        numerator / denominator,  # Value if true
        np.nan  # Value if false (e.g., set to NaN if denominator is 0)
        # You could also put 0 or a very large number here depending on how you want to treat it
    )

In [461]:
condition1 = (df_filtered['ols_diff'] >= -0.009)
condition2 = (df_filtered['highlight_status'] != 'Other')
combined_condition_mask = condition1 & condition2
df_filtered.loc[combined_condition_mask, 'highlight_status'] = df_filtered.loc[combined_condition_mask, 'highlight_status'].astype(str) + '_refined'

In [462]:
if df_filtered.empty:
    print(f"DataFrame пуст после фильтрации с MIN_COMPLETED_ORDERS = {MIN_COMPLETED_ORDERS}. График не будет построен.")
else:
    fig = px.scatter(df_filtered,
                     x='bad_bids_share',
                     y='rides_badbids_share',
                     color='alpha_value_str',
                     symbol='highlight_status',
                     symbol_map={
                         'veracrus': 'star',
                         'veracrus_candidates': 'cross'
                     },
                     hover_data=df_filtered.columns.tolist(),
                     title='Impact of Alpha (Color) & City Highlight (Symbol) with OLS through Origin',
                     labels={
                         'share_of_bad_bids': 'Share of Bad Bids',
                         'share_done_orders_with_bad_tender': 'Share of Completed Orders with Bad Tender',
                         'alpha_value_str': 'Alpha Value',
                         'highlight_status': 'City Status'
                     }
                    )

    alpha_color_map = {}
    for trace in fig.data:
        alpha_val_for_trace = trace.legendgroup
        if alpha_val_for_trace not in alpha_color_map:
            alpha_color_map[alpha_val_for_trace] = trace.marker.color
    print(alpha_color_map)

    unique_alpha_values = sorted(df_filtered['alpha_value_str'].unique())

    for current_alpha_str in unique_alpha_values:
        # Данные для OLS берутся по ВСЕМ точкам для текущего current_alpha_str
        group_data_for_ols = df_filtered[df_filtered['alpha_value_str'] == current_alpha_str]

        x_data = group_data_for_ols['bad_bids_share']
        y_data = group_data_for_ols['rides_badbids_share']

        valid_indices = x_data.notna() & y_data.notna()
        x_clean = x_data[valid_indices]
        y_clean = y_data[valid_indices]

        if len(x_clean) >= 2:
            try:
                model = sm.OLS(y_clean, x_clean)
                results = model.fit()

                if results.params.empty:
                    print(f"Предупреждение: Параметры OLS пусты для alpha_value_str = {current_alpha_str}.")
                    continue
                
                slope = results.params.iloc[0]

                if pd.notna(slope) and np.isfinite(slope):
                    x_range_for_line = np.array([-0.05, 1.0])
                    y_line = slope * x_range_for_line
                    
                    line_color = get_color_for_alpha(current_alpha_str, alpha_color_map)
                    if line_color is None:
                        print(f"Предупреждение: Цвет не найден для OLS alpha_value_str = {current_alpha_str}. Используется черный.")
                        line_color = 'rgba(0,0,0,0.5)'

                    fig.add_trace(go.Scatter(
                        x=x_range_for_line,
                        y=y_line,
                        mode='lines',
                        name=f'OLS Trend ({current_alpha_str})',
                        line=dict(color=line_color, width=2, dash='dash'),
                        legendgroup=f"group_{current_alpha_str}", 
                        showlegend=True
                    ))
                else:
                    print(f"Предупреждение: Наклон (slope) NaN/Inf для alpha_value_str = {current_alpha_str}.")
            except Exception as e:
                print(f"Ошибка OLS для alpha_value_str = {current_alpha_str}: {e}")
        else:
            print(f"Мало данных (нужно >= 2, есть {len(x_clean)}) для OLS alpha_value_str = {current_alpha_str}.")

    fig.update_layout(
        xaxis=dict(range=[-0.05, 1]),
        yaxis=dict(range=[-0.05, 1]),
        height=1000,
        legend_title_text="Legend"
    )

    fig.show()

{'0.1, Other': '#636efa', '0.1, LowRight_refined': '#636efa', '0.1, Middle_refined': '#636efa', '0.1, Middle': '#636efa', '0.1, Recife_like': '#636efa', '0.1, Recife_like_refined': '#636efa', '0.1, LowRight': '#636efa', '0.125, Other': '#EF553B', '0.125, LowRight_refined': '#EF553B', '0.125, Middle_refined': '#EF553B', '0.125, Middle': '#EF553B', '0.125, Recife_like': '#EF553B', '0.125, Recife_like_refined': '#EF553B', '0.125, LowRight': '#EF553B', '0.15, Other': '#00cc96', '0.15, LowRight_refined': '#00cc96', '0.15, Middle_refined': '#00cc96', '0.15, Middle': '#00cc96', '0.15, Recife_like': '#00cc96', '0.15, Recife_like_refined': '#00cc96', '0.15, LowRight': '#00cc96', '0.2, Other': '#ab63fa', '0.2, LowRight_refined': '#ab63fa', '0.2, Middle_refined': '#ab63fa', '0.2, Middle': '#ab63fa', '0.2, Recife_like': '#ab63fa', '0.2, Recife_like_refined': '#ab63fa', '0.25, Other': '#FFA15A', '0.25, LowRight_refined': '#FFA15A', '0.25, Middle_refined': '#FFA15A', '0.25, Middle': '#FFA15A', '0.25

In [469]:
df_filtered[df_filtered['city_id'].isin([4142, 4376])].sort_values(by='city_id', ascending=True)

,city_id,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status,ols_predicted_value,ols_diff,rel_diff
3008,4142,auto_econom,370.297701,0.100,8124448.0,3911505.0,0.481449,1310517.0,333752.0,0.254672,0.1,Recife_like,0.240554,-0.014118,0.058690
3009,4142,auto_econom,370.297701,0.125,8124448.0,3550080.0,0.436963,1310517.0,281123.0,0.214513,0.125,Recife_like,0.200993,-0.013520,0.067267
3010,4142,auto_econom,370.297701,0.150,8124448.0,3257332.0,0.400930,1310517.0,245886.0,0.187625,0.15,Recife_like,0.178470,-0.009155,0.051297
3011,4142,auto_econom,370.297701,0.200,8124448.0,2650367.0,0.326221,1310517.0,182308.0,0.139112,0.2,Recife_like,0.121895,-0.017216,0.141236
3012,4142,auto_econom,370.297701,0.250,8124448.0,2058450.0,0.253365,1310517.0,125248.0,0.095571,0.25,Recife_like,0.077637,-0.017934,0.231000
3013,4142,auto_econom,370.297701,0.300,8124448.0,1566517.0,0.192815,1310517.0,75904.0,0.057919,0.3,Recife_like_refined,0.049288,-0.008631,0.175112
3014,4142,auto_econom,370.297701,0.350,8124448.0,1273785.0,0.156784,1310517.0,60855.0,0.046436,0.35,Recife_like,0.030376,-0.016060,0.528713
3015,4142,auto_econom,370.297701,0.400,8124448.0,927949.0,0.114217,1310517.0,40603.0,0.030982,0.4,Recife_like,0.019293,-0.011689,0.605849
5320,4376,auto_econom,339.835542,0.100,2657623.0,1361395.0,0.512260,462789.0,124845.0,0.269767,0.1,Recife_like,0.255949,-0.013818,0.053986
5321,4376,auto_econom,339.835542,0.125,2657623.0,1223713.0,0.460454,462789.0,100106.0,0.216310,0.125,Recife_like_refined,0.211798,-0.004512,0.021303


In [464]:
condition1 = (df_filtered['highlight_status'] == 'LowRight_refined')
condition2 = (df_filtered['bad_bids_share'] >= 0.178)
combined_condition_mask = condition1 & condition2
lowright_res = df_filtered[combined_condition_mask].copy()
lowright_res['highlight_status'] = 'LowRight_Cluster'

condition1 = (df_filtered['highlight_status'] == 'Recife_like_refined')
condition2 = (df_filtered['bad_bids_share'] >= 0.19)
combined_condition_mask = condition1 & condition2
recife_res = df_filtered[combined_condition_mask].copy()
recife_res['highlight_status'] = 'Recife_Cluster'

condition1 = (df_filtered['highlight_status'] == 'Middle_refined')
condition2 = (df_filtered['bad_bids_share'] >= 0.17)
combined_condition_mask = condition1 & condition2
middle_res = df_filtered[combined_condition_mask].copy()
middle_res['highlight_status'] = 'Middle_Cluster'

In [465]:
v2_result = pd.concat([lowright_res, recife_res, middle_res], ignore_index=True)
v2_result = pd.merge(names_df, v2_result, on='city_id', how='right')
v2_result = v2_result.iloc[:, :-3]
v2_result

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,5957,Kathmandu,Nepal,auto_econom,454.014028,0.100,6753095.0,2500180.0,0.370227,986776.0,186249.0,0.188745,0.1,LowRight_Cluster
1,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.100,3211709.0,753594.0,0.234640,666460.0,68028.0,0.102074,0.1,LowRight_Cluster
2,5957,Kathmandu,Nepal,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,LowRight_Cluster
3,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,0.082680,0.125,LowRight_Cluster
4,5957,Kathmandu,Nepal,auto_econom,454.014028,0.150,6753095.0,1562347.0,0.231353,986776.0,110088.0,0.111563,0.15,LowRight_Cluster
5,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.150,3211709.0,574329.0,0.178823,666460.0,47277.0,0.070937,0.15,LowRight_Cluster
6,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.100,6469584.0,2803966.0,0.433407,999027.0,209222.0,0.209426,0.1,Recife_Cluster
7,4376,Curitiba,Brazil,auto_econom,339.835542,0.125,2657623.0,1223713.0,0.460454,462789.0,100106.0,0.216310,0.125,Recife_Cluster
8,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.125,6469584.0,2329211.0,0.360025,999027.0,144755.0,0.144896,0.125,Recife_Cluster
9,4376,Curitiba,Brazil,auto_econom,339.835542,0.150,2657623.0,1122240.0,0.422272,462789.0,87410.0,0.188877,0.15,Recife_Cluster


In [466]:
selected_rows_df_veracrus

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,4381,Monclova,Mexico,auto_econom,220.618175,0.10,1179737.0,138486.0,0.117387,459629.0,19135.0,0.041631,0.1,Veracruz_Cluster
1,4271,Iquique y Alto Hospicio,Chile,auto_econom,200.416556,0.15,1901412.0,253299.0,0.133216,594691.0,26782.0,0.045035,0.15,Veracruz_Cluster
2,4244,Cartagena,Colombia,auto_econom_nocomment,270.369901,0.20,3142921.0,577197.0,0.183650,869506.0,48611.0,0.055906,0.2,Veracruz_Cluster
3,5548,Panama,Panama,auto_econom,270.392303,0.20,4609340.0,692657.0,0.150272,1348288.0,51635.0,0.038297,0.2,Veracruz_Cluster
4,5504,Cochabamba,Bolivia,auto_econom,208.292639,0.20,2315121.0,362965.0,0.156780,671074.0,34667.0,0.051659,0.2,Veracruz_Cluster


In [467]:
selected_rows_df_puerto


,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,4227,Ciudad Juarez,Mexico,auto_econom,311.244127,0.125,3771375.0,1071000.0,0.283981,974039.0,134341.0,0.137922,0.125,Puerto_Cluster
1,262,Павлодар,Казахстан,auto_econom,208.435669,0.150,1216634.0,303646.0,0.249579,314198.0,34024.0,0.108288,0.15,Puerto_Cluster
2,235,Тараз,Казахстан,auto_econom,236.561399,0.150,1381775.0,405628.0,0.293556,338141.0,47718.0,0.141119,0.15,Puerto_Cluster
3,4227,Ciudad Juarez,Mexico,auto_econom,311.244127,0.150,3771375.0,931071.0,0.246878,974039.0,119546.0,0.122732,0.15,Puerto_Cluster
4,4240,Mazatlan,Mexico,auto_econom,251.085345,0.150,1754872.0,440479.0,0.251003,455620.0,46709.0,0.102517,0.15,Puerto_Cluster
5,228,Семей,Казахстан,auto_econom,201.728708,0.150,2106598.0,599530.0,0.284596,511413.0,68088.0,0.133137,0.15,Puerto_Cluster
6,4231,San Luis Potosi,Mexico,auto_econom,275.777859,0.200,3088359.0,651942.0,0.211097,904724.0,89455.0,0.098875,0.2,Puerto_Cluster
7,4194,Merida,Mexico,auto_econom,272.297612,0.200,4746830.0,980241.0,0.206504,1247554.0,127517.0,0.102214,0.2,Puerto_Cluster


In [468]:
final_list = pd.concat([v2_result, selected_rows_df_veracrus, selected_rows_df_puerto], ignore_index=True)
final_list.to_csv('final_list.csv', index=False)
final_list

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status
0,5957,Kathmandu,Nepal,auto_econom,454.014028,0.100,6753095.0,2500180.0,0.370227,986776.0,186249.0,0.188745,0.1,LowRight_Cluster
1,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.100,3211709.0,753594.0,0.234640,666460.0,68028.0,0.102074,0.1,LowRight_Cluster
2,5957,Kathmandu,Nepal,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,LowRight_Cluster
3,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,0.082680,0.125,LowRight_Cluster
4,5957,Kathmandu,Nepal,auto_econom,454.014028,0.150,6753095.0,1562347.0,0.231353,986776.0,110088.0,0.111563,0.15,LowRight_Cluster
5,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.150,3211709.0,574329.0,0.178823,666460.0,47277.0,0.070937,0.15,LowRight_Cluster
6,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.100,6469584.0,2803966.0,0.433407,999027.0,209222.0,0.209426,0.1,Recife_Cluster
7,4376,Curitiba,Brazil,auto_econom,339.835542,0.125,2657623.0,1223713.0,0.460454,462789.0,100106.0,0.216310,0.125,Recife_Cluster
8,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.125,6469584.0,2329211.0,0.360025,999027.0,144755.0,0.144896,0.125,Recife_Cluster
9,4376,Curitiba,Brazil,auto_econom,339.835542,0.150,2657623.0,1122240.0,0.422272,462789.0,87410.0,0.188877,0.15,Recife_Cluster


### !

In [370]:
final_list['order_type'].unique().tolist()

['auto_econom',
 'auto_econom_dublicate',
 'auto_econom_nocomment',
 'auto_econom_wheelchair']

In [366]:
from src.otherbid_share import calculate_average_option_3_plus_share

In [ ]:
shares_df = calculate_average_option_3_plus_share(city_ids=final_list['city_id'].unique().tolist(),
                                                   order_types=final_list['order_type'].unique().tolist(),
                                                   start_date='2025-05-11',
                                                   end_date='2025-05-17',
                                                   print_query=True)
shares_df.to_csv('buttons_shares.csv', index=False)

--- Generated SQL Query ---

WITH
timezone_data AS (
    SELECT 
        city_id, 
        timezone
    FROM `indriver-e6e40.emart.incity_detail`
    WHERE 
        city_id IN UNNEST([5957, 4208, 9417, 4376, 4197, 9415, 4142, 4243, 4244, 4253, 262, 4267, 4271, 238, 4258, 228, 4381, 5548, 5504, 4227, 235, 4240, 4231, 4194])
        AND created_date_order_part BETWEEN DATE_SUB(PARSE_DATE('%Y-%m-%d', '2025-05-11'), INTERVAL 7 DAY) 
                                      AND PARSE_DATE('%Y-%m-%d', '2025-05-17') 
    QUALIFY ROW_NUMBER() OVER (PARTITION BY city_id ORDER BY created_date_order_part DESC) = 1
),

rider_data AS (
    SELECT 
        t1.city_id,
        t1.uuid AS order_uuid,
        payment_price_value AS price,
        t1.modified_at AS modified_at_utc,
        'rider_price' AS price_type,
        price_highrate_value,
        CAST(NULL AS INT64) AS order_rn,
        CAST(NULL AS INT64) AS tender_rn,
        CAST(NULL AS STRING) AS tender_uuid,
        CAST(NULL AS ARRAY<INT64>

KeyboardInterrupt: 

In [368]:
shares_df = pd.read_csv((f'data/buttons_shares.csv'))
pd.merge(v2_result, shares_df, on=['city_id', 'order_type'], how='left')

,city_id,city,country,order_type,avg_rta,alpha_value,bids_count,bad_bids_count,bad_bids_share,rides_count,rides_badbids_count,rides_badbids_share,alpha_value_str,highlight_status,avg_share_option_3_plus_bids
0,5957,Kathmandu,Nepal,auto_econom,454.014028,0.100,6753095.0,2500180.0,0.370227,986776.0,186249.0,0.188745,0.1,LowRight_Cluster,0.000000
1,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.100,3211709.0,753594.0,0.234640,666460.0,68028.0,0.102074,0.1,LowRight_Cluster,0.125239
2,5957,Kathmandu,Nepal,auto_econom,454.014028,0.125,6753095.0,1936053.0,0.286691,986776.0,133031.0,0.134814,0.125,LowRight_Cluster,0.000000
3,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.125,3211709.0,645938.0,0.201120,666460.0,55103.0,0.082680,0.125,LowRight_Cluster,0.125239
4,5957,Kathmandu,Nepal,auto_econom,454.014028,0.150,6753095.0,1562347.0,0.231353,986776.0,110088.0,0.111563,0.15,LowRight_Cluster,0.000000
5,4208,Rabat (الرباط),Morocco,auto_econom_dublicate,361.098065,0.150,3211709.0,574329.0,0.178823,666460.0,47277.0,0.070937,0.15,LowRight_Cluster,0.125239
6,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.100,6469584.0,2803966.0,0.433407,999027.0,209222.0,0.209426,0.1,Recife_Cluster,0.150456
7,4376,Curitiba,Brazil,auto_econom,339.835542,0.125,2657623.0,1223713.0,0.460454,462789.0,100106.0,0.216310,0.125,Recife_Cluster,0.000000
8,9417,Alger (الجزائر العاصمة),Algeria,auto_econom,716.872535,0.125,6469584.0,2329211.0,0.360025,999027.0,144755.0,0.144896,0.125,Recife_Cluster,0.150456
9,4376,Curitiba,Brazil,auto_econom,339.835542,0.150,2657623.0,1122240.0,0.422272,462789.0,87410.0,0.188877,0.15,Recife_Cluster,0.000000
